<a href="https://colab.research.google.com/github/Bayrkdartech/Binder/blob/main/PMJE_MW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# استيراد المكتبات الأساسية
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
import os
from google.colab import drive

# 1. توصيل Google Drive
drive.mount('/content/drive')

# 2. إعداد مجلد kaggle
os.makedirs("/root/.kaggle", exist_ok=True)

# 3. نسخ ملف kaggle.json من Google Drive إلى مجلد kaggle
# غيّر المسار إذا ملف kaggle.json موجود بمكان آخر
!cp /content/drive/MyDrive/kaggle.json /root/.kaggle/

# 4. ضبط الأذونات
!chmod 600 /root/.kaggle/kaggle.json

# 5. إنشاء مجلد data إذا غير موجود
os.makedirs("data", exist_ok=True)

# 6. تحميل البيانات من Kaggle إذا غير موجودة
if not os.path.exists("data/hourly-energy-consumption.zip"):
    !kaggle datasets download -d robikscube/hourly-energy-consumption -p data -q
else:
    print("📂 الملف موجود مسبقًا ولن يتم إعادة التحميل.")

# 7. فك الضغط (بدون رسائل)
if not os.path.exists("data/AEP_hourly.csv"):  # نتأكد إذا تفك الضغط سابقًا
    !unzip -q data/hourly-energy-consumption.zip -d data
else:
    print("✅ الملفات مفكوكة مسبقًا.")

# 8. قراءة ملف CSV مثال
df = pd.read_csv("data/AEP_hourly.csv")
print("أول 5 صفوف من البيانات:")
print(df.head())


# FIRST STEP ###


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📂 الملف موجود مسبقًا ولن يتم إعادة التحميل.
✅ الملفات مفكوكة مسبقًا.
أول 5 صفوف من البيانات:
              Datetime   AEP_MW
0  2004-12-31 01:00:00  13478.0
1  2004-12-31 02:00:00  12865.0
2  2004-12-31 03:00:00  12577.0
3  2004-12-31 04:00:00  12517.0
4  2004-12-31 05:00:00  12670.0
